In [1]:
from openai import OpenAI
api_key = '0' # i will give you the key later
client = OpenAI(api_key=api_key)

In [21]:
def readfile(path):
    with open(path, 'r') as file:
        return file.read()
docu =readfile("envision-brief.md")    
challenge = readfile("mychallenges/c"+"000"+".md")

# decompose challenge into question and prof_answer
def decompose_challenge(challenge):
    question,prof_answer=challenge.split("\n\n# ANSWER\n\n")
    return question, prof_answer
question,prof_answer=decompose_challenge(challenge)
print(question,prof_answer)

define a text literal called greetings with value "Hello" and display greetings on the dashboard as a label.   ```envision
greeting = "Hello" // define the text literal
show label greeting // show the text literal as a label
```


In [23]:
coder_personality="You are a proficient coder in the Domain Specific Language called Envision. \
    Your task is to generate response to the given challenge. \
    Some challenges will ask you to generate Envision code,\
    others will ask you to explain given code or answer questions related to the Envision language. \
    Do not output any intermediate thinking, only give the final answer.\
    Here is the documentation of Envision:\
    ### Documentation\n" + docu
coder_prompt=question

coder_response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": coder_personality},
        {"role": "user", "content": coder_prompt}
    ],
    max_tokens=1000,  # Adjust the number of tokens based on your needs
    temperature=0.2,
)
stud_sentence=coder_response.choices[0].message.content
print(stud_sentence)


```envision
greetings = "Hello"
show label "" a1b1 with greetings
```


In [47]:

# i propose 2 personalities for the judge
# 1st sticks to prior knowledge (eg from python and SQL), 2nd sticks more to professor's answer
judge_personality_knowledgeAuthority="Your goal is to judge whether the STUDENT ANSWER is OK or NOT_OK, as an answer to the QUESTION.\
In order to judge the STUDENT ANSWER, you are given the PROFESSOR ANSWER which is always OK.\
Difference in details like variable names should be tolerated, as long as it is not specifically demanded in the QUESTION. \
Let's think aloud step by step to help your judgement. \
At the end of your thinking, you must add a new line which contains either OK or NOT_OK and nothing more."
judge_personality_teacherAuthority="Your goal is to judge whether the STUDENT ANSWER is OK or NOT_OK, as an answer to the QUESTION.\
In order to judge the STUDENT ANSWER, you are given the PROFESSOR ANSWER which is always OK.\
Even if the STUDENT ANSWER is not correct by our prior knowledge, we should consider it OK as long as\
it corresponds to the PROFESSOR ANSWER.\
Difference in details like variable names should be tolerated, as long as it is not specifically demanded in the QUESTION. \
Let's think aloud step by step to help your judgement. \
At the end of your thinking, you must add a new line which contains either OK or NOT_OK and nothing more."

judge_prompt = "### QUESTION: "+question+"\n### PROFESSOR ANSWER: "+prof_answer+"\n### STUDENT ANSWER: "+stud_sentence

# Generate a response from the chatbot
judge_response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": judge_personality_teacherAuthority},
        {"role": "user", "content": judge_prompt}
    ],
    max_tokens=800,  # Adjust the number of tokens based on your needs
    temperature=0.2,
)
print(judge_prompt)
# Print the generated response
print(judge_response.choices[0].message.content)


### QUESTION: define a text literal called greetings with value "Hello" and display greetings on the dashboard as a label.  
### PROFESSOR ANSWER: ```envision
greeting = "Hello" // define the text literal
show label greeting // show the text literal as a label
```
### STUDENT ANSWER: ```envision
greetings = "Hello"
show label "" a1b1 with greetings
```
The student's answer defines a text literal called greetings with the value "Hello" and displays it on the dashboard as a label. The variable name used by the student is slightly different from the professor's answer, but it still corresponds to the task. The student also correctly shows the label with the variable greetings. Therefore, the student's answer is OK.  

OK


In [25]:
# extract the 'real' code from the student answer (cut away the '''envision bit at the start and end)
def extract_code(stud_sentence):
    lines = stud_sentence.strip().split('\n')
    return '\n'.join(lines[1:-1])
print(extract_code(stud_sentence))

greetings = "Hello"
show label "" a1b1 with greetings


In [43]:
import requests
# send code to online compiler and check if it compiles

def check_compilation(script):
    url = "https://try.lokad.com/w/script/trycompile"
    payload = {
        "Script": script
    }

    try:
        # Send POST request
        response = requests.post(url, json=payload)

        # Check for successful response
        if response.status_code == 200:
            result = response.json()
            if result["IsCompOk"]:
                return True
            else:
                print("Compilation Failed!")
                for message in result["CompMessages"]:
                    print(f"Error: {message['Text']} (Line: {message['Line']}, Start: {message['Start']}, Length: {message['Length']}, Severity: {message['Severity']})")
                    return False
        else:
            print("Error: Unable to reach the compilation service.")
            return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

        

# Example usage
check_compilation(extract_code(stud_sentence))


Compilation Failed!
Error: 'show %s' expects no 'with'. (Line: 2, Start: 20, Length: 4, Severity: Error)


False

In [44]:
# for each question, try 3 generation-compilations.
# if compiles, further check with judge.
def pipeline_verify(challenge,coder_personality,judge_personality=judge_personality_teacherAuthority):
    question,prof_answer=decompose_challenge(challenge)
    n_tries=3
    for compile_try in range(n_tries):
        coder_prompt=question
        coder_response = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=[
                {"role": "system", "content": coder_personality},
                {"role": "user", "content": coder_prompt}
            ],
            max_tokens=1000,  # Adjust the number of tokens based on your needs
            temperature=0.2,
        )
        stud_sentence=coder_response.choices[0].message.content
        if(check_compilation(extract_code(stud_sentence))):
            print('compile ok')
            break
        elif (compile_try==n_tries-1):
            print( "too many failures !")
            print(extract_code(stud_sentence))
            return stud_sentence,"too many failures !",False

    judge_prompt = "### QUESTION: "+question+"\n### PROFESSOR ANSWER: "+prof_answer+"\n### STUDENT ANSWER: "+stud_sentence
    judge_response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": judge_personality},
            {"role": "user", "content": judge_prompt}
        ],
        max_tokens=800,  # Adjust the number of tokens based on your needs
        temperature=0.2,
    )
    judge_sentence=judge_response.choices[0].message.content
    judge_decision=judge_sentence.split("\n")[-1]
    print ('judge_decision:',judge_decision)
    return stud_sentence,judge_sentence,(judge_decision=='OK')
# a all-in-one function to score a model on a list of challenges
def pipeline_score_allchallenge(challenges,coder_personality):
    score=0
    for challenge in challenges:
        _,_,judge_decision=pipeline_verify(challenge,coder_personality)
        if (judge_decision): score+=1
    print('correct:'+str(score)+' out of '+str(len(challenges))+', '+str(score/len(challenges)*100)+'%')  

In [48]:
# test the pipeline. typically fails on challenges 0,4,7 
# (these questions include grammar rules not covered in the documentation)

# do not use 005(which is not a question of code generation, does not fit in the pipeline format.)

indexes=["000","001","002","003","004","006","007","008"]
challenges=[readfile("mychallenges/c"+index+".md") for index in indexes]
pipeline_score_allchallenge(challenges,coder_personality)

Compilation Failed!
Error: 'show %s' expects no 'with'. (Line: 2, Start: 20, Length: 4, Severity: Error)
Compilation Failed!
Error: 'show %s' expects no 'with'. (Line: 5, Start: 20, Length: 4, Severity: Error)
Compilation Failed!
Error: 'show %s' expects no 'with'. (Line: 5, Start: 20, Length: 4, Severity: Error)
too many failures !
// Define a text literal called greetings with value "Hello"
greetings = "Hello"

// Display greetings on the dashboard as a label
show label "" a1b2 with greetings
compile ok
judge_decision: OK
compile ok
judge_decision: OK
compile ok
judge_decision: OK
Compilation Failed!
Error: Found identifier but expected 'upload', 'table', tile-type or 'chart'. (Line: 4, Start: 5, Length: 1, Severity: Error)
Compilation Failed!
Error: Found identifier but expected 'upload', 'table', tile-type or 'chart'. (Line: 4, Start: 5, Length: 1, Severity: Error)
Compilation Failed!
Error: Found identifier but expected 'upload', 'table', tile-type or 'chart'. (Line: 4, Start: 5, 